<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_YOLOv8m_Focal_480_052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8-m Classification @ 480px on Colab

Make sure you’ve set your Runtime ▶️ “Change runtime type” ▶️ GPU.

In [ ]:
# @title 1️⃣ Install dependencies
# Colab comes with torch+cuda preinstalled, but we upgrade ultralytics.
!pip install --upgrade ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.6 MB/s eta 0:00:00


In [ ]:
# @title 2️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title 3️⃣ Paths & Unzip Dataset
import os, zipfile, glob
from pathlib import Path

# Adjust these to match where you put your ZIP in Drive:
ZIP_PATH     = "/content/drive/MyDrive/Split_mixed_newaugmented_480.zip"
EXTRACT_DIR  = "/content/dataset480"
SPLIT_DIR    = os.path.join(EXTRACT_DIR, "Split_480")  # must contain train/ & val/
RESULTS_DIR  = "/content/drive/MyDrive/yolov8m_cls_480_results"

# 3.1 Unzip if needed
if not os.path.isdir(SPLIT_DIR):
    print("📦 Unzipping dataset…")
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(EXTRACT_DIR)
    print("✅ Unzipped to", EXTRACT_DIR)

# 3.2 Verify
assert os.path.isdir(os.path.join(SPLIT_DIR, "train")), "❌ train/ missing!"
assert os.path.isdir(os.path.join(SPLIT_DIR, "val"  )), "❌ val/   missing!"

train_images = glob.glob(os.path.join(SPLIT_DIR, "train", "*", "*.jpg"))
val_images   = glob.glob(os.path.join(SPLIT_DIR, "val",   "*", "*.jpg"))
print(f"🧾 Found {len(train_images):,} train and {len(val_images):,} val images")

📦 Unzipping dataset…
✅ Unzipped to /content/dataset480
🧾 Found 156,504 train and 17,276 val images


In [ ]:
# @title 4️⃣ Compute α (class imbalance) for focal loss
import torch

# your class counts
class_counts = {
  "Control_Panel":11700, "Drinking":11700, "Eating":2689,   "Makeup":11700,
  "Normal":12196,        "Phone_Call_(Left)":11700, "Phone_Call_(Right)":11700,
  "Reaching_Behind":11700,"Sleep":12232,  "Smoking":12376,
  "Talk_to_passengers":11700,"Text_(Left)":11700, "Text_(Right)":11700,
  "Yawning":11700,
}

max_n = max(class_counts.values())
alpha = torch.tensor([max_n / class_counts[c] for c in class_counts],
                     dtype=torch.float32).cuda()
print("✅ α weights:", alpha)

✅ α weights: tensor([1.0578, 1.0578, 4.6025, 1.0578, 1.0148, 1.0578, 1.0578, 1.0578, 1.0118,
        1.0000, 1.0578, 1.0578, 1.0578, 1.0578], device='cuda:0')


In [ ]:
# @title 5️⃣ Define FocalLoss & Train YOLOv8-m-cls
import torch.nn as nn
from ultralytics import YOLO

# ─── Focal Loss ──────────────────────────
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super().__init__()
        self.gamma, self.alpha, self.reduction = gamma, alpha, reduction

    def forward(self, inputs, targets):
        logp = torch.nn.functional.log_softmax(inputs, dim=-1)
        p    = torch.exp(logp)
        t    = targets.long()
        pt   = p.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        focal = (1 - pt) ** self.gamma
        loss = -focal * logp.gather(-1, t.unsqueeze(-1)).squeeze(-1)
        if self.alpha is not None:
            loss = loss * self.alpha[t]
        return loss.mean() if self.reduction=='mean' else loss.sum()

# ─── Load & override loss ─────────────────
print("🚀 Loading YOLOv8m-cls…")
# model = YOLO("yolov8m-cls.pt")             # pretrained backbone
model = YOLO('/content/drive/MyDrive/yolov8m_cls_480_results/yolov8m_cls_480_focal/weights/last.pt')
model.loss = FocalLoss(gamma=2.0, alpha=alpha)

# ─── Train ───────────────────────────────
print("📚 Training…")
model.train(
    data=SPLIT_DIR,          # train/ & val/ subfolders
    epochs=50,
    batch=32,
    imgsz=480,
    project=RESULTS_DIR,
    name="yolov8m_cls_480_focal",
    pretrained=False,
    lr0=5e-4,
    optimizer="Adam",
    resume=True,
)
print("✅ Training resumed!")


# ─── Validate & copy best.pt ────────────
print("📊 Validating…")
model.val()

src = os.path.join(model.trainer.save_dir, "weights/best.pt")
dst = os.path.join(RESULTS_DIR, "yolov8m_cls_480_focal", "weights/best.pt")
os.makedirs(os.path.dirname(dst), exist_ok=True)
os.system(f"cp {src} {dst}")
print("✅ Best weights saved to:", dst)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Loading YOLOv8m-cls…
📚 Training…
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset480/Split_480, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8

100%|██████████| 5.35M/5.35M [00:00<00:00, 96.7MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1563.8±554.5 MB/s, size: 43.7 KB)


train: Scanning /content/dataset480/Split_480/train... 156504 images, 0 corrupt: 100%|██████████| 156504/156504 [00:43<00:00, 3585.25it/s]


train: New cache created: /content/dataset480/Split_480/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1236.0±444.2 MB/s, size: 47.2 KB)


...

               classes   top1_acc   top5_acc:   2%|▏         | 6/270 [00:00<00:38,  6.87it/s]

               classes   top1_acc   top5_acc:   4%|▍         | 11/270 [00:01<00:34,  7.41it/s]
100%|██████████| 755k/755k [00:00<00:00, 23.5MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 270/270 [00:36<00:00,  7.44it/s]


                   all      0.995          1
Speed: 0.3ms preprocess, 1.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_480_results/yolov8m_cls_480_focal
✅ Training resumed!
📊 Validating…
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,780,590 parameters, 0 gradients, 41.6 GFLOPs
train: /content/dataset480/Split_480/train... found 156504 images in 14 classes ✅ 
val: /content/dataset480/Split_480/val... found 17276 images in 14 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1952.5±676.3 MB/s, size: 47.2 KB)


val: Scanning /content/dataset480/Split_480/val... 17276 images, 0 corrupt: 100%|██████████| 17276/17276 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 540/540 [00:59<00:00,  9.12it/s]


                   all      0.995          1
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8m_cls_480_results/yolov8m_cls_480_focal2
✅ Best weights saved to: /content/drive/MyDrive/yolov8m_cls_480_results/yolov8m_cls_480_focal/weights/best.pt


In [ ]:
import os
import time
from google.colab import runtime

# Wait for training to complete before disconnecting
print("Training completed! Releasing GPU resources...")
time.sleep(20)  # Give some time for final processing

# Automatically disconnect the Colab session
runtime.unassign()
os._exit(0)

Training completed! Releasing GPU resources...
